In [1]:
!pip install --upgrade pydantic
!pip uninstall pydantic-core -y
!pip install pydantic-core
from pydantic import ValidationError

#from bpmn_assistant.core.enums import BPMNElementType
#from bpmn_assistant.core.schemas import BPMNTask, ExclusiveGateway, ParallelGateway

from enum import Enum


class BPMNElementType(Enum):
    TASK = "task"
    USER_TASK = "userTask"
    SERVICE_TASK = "serviceTask"
    EXCLUSIVE_GATEWAY = "exclusiveGateway"
    PARALLEL_GATEWAY = "parallelGateway"
    START_EVENT = "startEvent"
    END_EVENT = "endEvent"
    
from typing import List, Optional, Union, Dict, Any

from pydantic import BaseModel
from typing_extensions import Literal

TaskType = Literal["task", "userTask", "serviceTask"]


class BPMNTask(BaseModel):
    """
    Represents a BPMN task.
    'type' must be one of: 'task', 'userTask', or 'serviceTask'.
    """

    type: TaskType
    id: str
    label: str


EventType = Literal["startEvent", "endEvent"]


class BPMNEvent(BaseModel):
    """
    Represents a BPMN event.
    'type' must be one of: 'startEvent', 'endEvent'.
    """

    type: EventType
    id: str


class ExclusiveGatewayBranch(BaseModel):
    """
    Represents a branch of an exclusive gateway.
    - 'condition': textual condition for the branch
    - 'path': array of BPMN elements executed if the condition is met
    - 'next': optional ID of the next element (if not following default sequence)
    """

    condition: str
    path: List["BPMNElement"] = []
    next: Optional[str] = None


class ExclusiveGateway(BaseModel):
    """
    Represents a BPMN exclusive gateway.
    - 'has_join': indicates whether this gateway also merges paths
    - 'branches': list of exclusive branches
    """

    type: Literal["exclusiveGateway"]
    id: str
    label: str
    has_join: bool
    branches: List[ExclusiveGatewayBranch]


class ParallelGateway(BaseModel):
    """
    Represents a BPMN parallel gateway.
    - 'branches': an array of arrays, each of which holds a list of BPMN elements
      to be executed in parallel.
    """

    type: Literal["parallelGateway"]
    id: str
    branches: List[List["BPMNElement"]]


BPMNElement = Union[BPMNTask, BPMNEvent, ExclusiveGateway, ParallelGateway]


class ProcessModel(BaseModel):
    """
    Represents a BPMN process containing a list of elements
    that can be tasks, events, or gateways.
    """

    process: List[BPMNElement]


#class EditProposal(BaseModel):
    """
    Represents an edit proposal for a BPMN process.
    """

    #function: str
    #arguments: Dict[str, Any]


#class StopSignal(BaseModel):
    """
    Represents a stop signal for the BPMN editing process.
    """

    #stop: Literal[True]

#IntermediateEditProposal = BaseModel[Union[EditProposal, StopSignal]]

def validate_bpmn(process: list) -> None:
    """
    Validate the BPMN process.
    Args:
        process: The BPMN process in JSON format.
    Raises:
        ValueError: If the BPMN process, or any of its elements, is invalid.
    """
    seen_ids = set()
    for element in process:
        validate_element(element)
        
        if element["id"] in seen_ids:
            raise ValueError(f"Duplicate element ID found: {element['id']}")
        seen_ids.add(element["id"])

        if element["type"] == BPMNElementType.EXCLUSIVE_GATEWAY.value:
            for branch in element["branches"]:
                validate_bpmn(branch["path"])
        if element["type"] == BPMNElementType.PARALLEL_GATEWAY.value:
            for branch in element["branches"]:
                validate_bpmn(branch)


def validate_element(element: dict) -> None:
    """
    Validate the BPMN element.
    Args:
        element: The BPMN element in JSON format.
    Raises:
        ValueError: If the BPMN element is invalid.
    """
    if "id" not in element:
        raise ValueError(f"Element is missing an ID: {element}")
    elif "type" not in element:
        raise ValueError(f"Element is missing a type: {element}")

    supported_elements = [e.value for e in BPMNElementType]

    if element["type"] not in supported_elements:
        raise ValueError(
            f"Unsupported element type: {element['type']}. Supported types: {supported_elements}"
        )

    if element["type"] in [
        BPMNElementType.TASK.value,
        BPMNElementType.USER_TASK.value,
        BPMNElementType.SERVICE_TASK.value,
    ]:
        _validate_task(element)

    elif element["type"] == BPMNElementType.EXCLUSIVE_GATEWAY.value:
        _validate_exclusive_gateway(element)

    elif element["type"] == BPMNElementType.PARALLEL_GATEWAY.value:
        _validate_parallel_gateway(element)


def _validate_task(element: dict) -> None:
    if "label" not in element:
        raise ValueError(f"Task element is missing a label: {element}")

    try:
        BPMNTask(**element)
    except ValidationError:
        raise ValueError(f"Invalid task element: {element}")


def _validate_exclusive_gateway(element: dict) -> None:
    if "label" not in element:
        raise ValueError(f"Exclusive gateway is missing a label: {element}")
    if "branches" not in element or not isinstance(element["branches"], list):
        raise ValueError(
            f"Exclusive gateway is missing or has invalid 'branches': {element}"
        )
    for branch in element["branches"]:
        if "condition" not in branch or "path" not in branch:
            raise ValueError(f"Invalid branch in exclusive gateway: {branch}")

    try:
        ExclusiveGateway(**element)
    except ValidationError:
        raise ValueError(f"Invalid exclusive gateway element: {element}")


def _validate_parallel_gateway(element: dict) -> None:
    if "branches" not in element or not isinstance(element["branches"], list):
        raise ValueError(
            f"Parallel gateway has missing or invalid 'branches': {element}"
        )

    try:
        ParallelGateway(**element)
    except ValidationError:
        raise ValueError(f"Invalid parallel gateway element: {element}")


Found existing installation: pydantic_core 2.41.5
Uninstalling pydantic_core-2.41.5:
  Successfully uninstalled pydantic_core-2.41.5
  Obtaining dependency information for pydantic-core from https://files.pythonhosted.org/packages/11/66/f14d1d978ea94d1bc21fc98fcf570f9542fe55bfcc40269d4e1a21c19bf7/pydantic_core-2.41.5-cp311-cp311-win_amd64.whl.metadata
  Using cached pydantic_core-2.41.5-cp311-cp311-win_amd64.whl.metadata (7.4 kB)
Using cached pydantic_core-2.41.5-cp311-cp311-win_amd64.whl (2.0 MB)


In [2]:
import json
import ast
from pathlib import Path


def load_process_from_file(path: str):
    """
    Lädt eine BPMN-Prozessstruktur aus einer .json- oder .txt-Datei.

    Unterstützte Formate:
    - JSON:
        {"process": [ ... ]}  oder  [ {...}, ... ]
    - TXT:
        - JSON (wie oben) oder
        - Python-Literal für Liste oder Dict (z.B. [ {...}, ... ])

    Rückgabewert:
        process: Liste von Element-Dicts, die an validate_bpmn(process) übergeben werden kann.
    """
    file_path = Path(path)
    if not file_path.exists():
        raise FileNotFoundError(f"Datei nicht gefunden: {file_path}")

    text = file_path.read_text(encoding="utf-8").strip()

    # 1) Erst versuchen wir JSON zu lesen
    try:
        data = json.loads(text)
    except json.JSONDecodeError:
        # 2) Wenn kein gültiges JSON: Python-Literal versuchen
        try:
            data = ast.literal_eval(text)
        except Exception as e:
            raise ValueError(
                f"Datei {file_path} ist weder gültiges JSON noch ein gültiges Python-Literal: {e}"
            )

    # 3) Aus data das process-Array machen
    if isinstance(data, dict) and "process" in data:
        process = data["process"]
    elif isinstance(data, list):
        process = data
    else:
        raise ValueError(
            f"Unerwartige Struktur in {file_path}: erwartet Dict mit 'process' oder Liste, aber bekommen: {type(data)}"
        )

    if not isinstance(process, list):
        raise ValueError(
            f"'process' in {file_path} ist kein Array, sondern: {type(process)}"
        )

    return process

def validate_process_file(path: str) -> None:
    """Lädt eine Datei, extrahiert den Prozess und validiert ihn mit validate_bpmn."""
    try:
        process = load_process_from_file(path)
    except Exception as e:
        print(f"❌ {path}: Konnte Prozess nicht laden:")
        print("   ", e)
        return

    try:
        validate_bpmn(process)
    except ValueError as e:
        # Hier merkst du: wir programmieren NICHT „erfolgreich“,
        # sondern reagieren je nachdem, ob validate_bpmn eine Exception wirft.
        print(f"⚠️ {path}: Prozess ist UNGÜLTIG:")
        print("   ", e)
    else:
        print(f"✅ {path}: Prozess ist gültig.")

In [6]:
process = load_process_from_file("Test.txt")
print(process)
#validate_bpmn(load_process_from_file("Invalid-example.txt"))
validate_process_file("Test.txt")

[{'type': 'startEvent', 'id': 'start', 'label': 'Purchase order received from customer'}, {'type': 'serviceTask', 'id': 'task1', 'label': 'Check if product is in stock'}, {'type': 'exclusiveGateway', 'id': 'gateway1', 'label': 'Is the product in stock?', 'has_join': False, 'branches': [{'condition': 'Product is in stock', 'path': [{'type': 'serviceTask', 'id': 'task2', 'label': 'Request product from warehouse'}, {'type': 'serviceTask', 'id': 'task3', 'label': 'Confirm order to customer'}]}, {'condition': 'Product is not in stock', 'path': [{'type': 'endEvent', 'id': 'end1', 'label': 'Order rejected due to lack of stock'}]}]}, {'type': 'parallelGateway', 'id': 'parallel1', 'branches': [[{'type': 'userTask', 'id': 'task4', 'label': 'Receive shipment address from customer'}, {'type': 'serviceTask', 'id': 'task5', 'label': 'Ship product to customer'}], [{'type': 'serviceTask', 'id': 'task6', 'label': 'Issue invoice to customer'}, {'type': 'serviceTask', 'id': 'task7', 'label': 'Receive pay